In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
import pandas as pd
#matplotlib 패키지 한글 깨짐 처리 시작
import matplotlib.pyplot as plt
import platform
if platform.system() == 'Darwin': #맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': #윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': #리눅스 (구글 콜랩)
        #!wget "https://www.wfonts.com/download/data/2016/06/13/malgun-gothic/malgun.ttf"
        #!mv malgun.ttf /usr/share/fonts/truetype/
        #import matplotlib.font_manager as fm 
        #fm._rebuild() 
        plt.rc('font', family='Malgun Gothic') 
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결
#matplotlib 패키지 한글 깨짐 처리 끝

In [3]:
data = pd.read_csv(r'C:\Users\user\Desktop\경진대회\금융보안원_경진대회_데이터(최종제출).csv')
data.head()

,기준년월,품목대분류코드,품목대분류명,품목중분류코드,품목중분류명,성별,연령,가구생애주기,고객소재지_광역시도,고객소재지_시군구,고객소재지_읍면동,매출금액,매출건수
0,201903,10,의류,1010,여성의류,여성,40대,초중고자녀가구,서울특별시,구로구,궁동,782521,13
1,201903,21,e상품/서비스,2110,o2o서비스,여성,20대,신혼영유아가구,서울특별시,구로구,궁동,963789,29
2,201903,19,출산/육아,1913,육아용품서비스,여성,30대,신혼영유아가구,서울특별시,구로구,궁동,924309,46
3,201903,21,e상품/서비스,2111,e머니/상품권,여성,30대,신혼영유아가구,서울특별시,구로구,궁동,1089360,54
4,201903,17,생활,1710,생활용품,여성,30대,신혼영유아가구,서울특별시,구로구,궁동,1656740,58


In [4]:
data = data.drop(['품목대분류명', '품목중분류명'], axis = 1)
data.head()

,기준년월,품목대분류코드,품목중분류코드,성별,연령,가구생애주기,고객소재지_광역시도,고객소재지_시군구,고객소재지_읍면동,매출금액,매출건수
0,201903,10,1010,여성,40대,초중고자녀가구,서울특별시,구로구,궁동,782521,13
1,201903,21,2110,여성,20대,신혼영유아가구,서울특별시,구로구,궁동,963789,29
2,201903,19,1913,여성,30대,신혼영유아가구,서울특별시,구로구,궁동,924309,46
3,201903,21,2111,여성,30대,신혼영유아가구,서울특별시,구로구,궁동,1089360,54
4,201903,17,1710,여성,30대,신혼영유아가구,서울특별시,구로구,궁동,1656740,58


In [5]:
data = data.drop(['기준년월', '품목중분류코드', '고객소재지_읍면동'], axis = 1)
data.head()

,품목대분류코드,성별,연령,가구생애주기,고객소재지_광역시도,고객소재지_시군구,매출금액,매출건수
0,10,여성,40대,초중고자녀가구,서울특별시,구로구,782521,13
1,21,여성,20대,신혼영유아가구,서울특별시,구로구,963789,29
2,19,여성,30대,신혼영유아가구,서울특별시,구로구,924309,46
3,21,여성,30대,신혼영유아가구,서울특별시,구로구,1089360,54
4,17,여성,30대,신혼영유아가구,서울특별시,구로구,1656740,58


In [6]:
def get_encoding(data, category, target):
    df = pd.DataFrame(data[category])
    for i in range(len(df)):
        if df.iloc[i, 0] == target : df.iloc[i, 0] = 1
        else: df.iloc[i, 0] = 0
    data.drop(category, axis = 1, inplace= True)
    data = data.join(df)
    return data

In [7]:
data = get_encoding(data, '품목대분류코드', 20)

In [ ]:
data.head()

In [ ]:
data['품목대분류코드'].value_counts()

In [ ]:
data_f = data[data['품목대분류코드'] == 1]

N = len(data_f)

category_list = ['성별', '연령', '가구생애주기']
fig, ax = plt.subplots(figsize = (16,8), ncols = len(category_list))
for i, category in enumerate(category_list):
    print(data_f[category].value_counts()/N*100)
    labels = data[category].unique()
    size = data_f[category].value_counts()/N*100
    ax[i].pie(size, labels = labels, autopct='%1.1f%%', shadow=True)
    ax[i].axis('equal')

In [ ]:
data_f_Seoul = data_f[data_f['고객소재지_광역시도'] == '서울특별시']
df = pd.DataFrame(data_f_Seoul['고객소재지_시군구'].value_counts())
df.sort_values(by = ['고객소재지_시군구'], axis=0, inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index' : '고객소재지_시군구', '고객소재지_시군구' : '합계'}, inplace=True)
df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (10, 8))
sns.barplot(x = '고객소재지_시군구', y = '합계', data = df)
fig.autofmt_xdate(rotation = 45)

In [ ]:
#인코딩 적용 함수 만들기
def get_dummies(column, df):
    dummies = pd.get_dummies(df[column])
    df.drop(column, axis = 1, inplace = True)
    df = df.join(dummies)
    return df

In [ ]:
columns = ['성별', '연령', '가구생애주기', '고객소재지_광역시도', '고객소재지_시군구']

for col in columns:
    data_dummies = get_dummies(col, data)

In [ ]:
data_dummies.head()

In [ ]:
N = len(data_dummies)
print(data_dummies['품목대분류코드'].value_counts()/N)

fig, ax = plt.subplots(1, 1, figsize = (12, 6))
sns.countplot(x = '품목대분류코드', data = data_dummies)

In [ ]:
def get_preprocessed_df(df, category):
    df_copy = df.copy()
    scaled = np.log1p(df_copy[category])
    df_copy.insert(0, category + '_Scaled', scaled)
    df_copy.drop(category, axis = 1, inplace = True)
    return df_copy

In [ ]:
data_dummies = get_preprocessed_df(data_dummies, '매출금액')

import warnings
warnings.filterwarnings('ignore')

fig, ax = plt.subplots(1, 1, figsize = (16, 8))
sns.distplot(data_dummies['매출금액_Scaled'])

In [ ]:
data_dummies = get_preprocessed_df(data_dummies, '매출건수')

import warnings
warnings.filterwarnings('ignore')

fig, ax = plt.subplots(1, 1, figsize = (16, 8))
sns.distplot(data_dummies['매출건수_Scaled'])

In [ ]:
df_M_and_C = data_dummies[['매출금액_Scaled','매출건수_Scaled']]

target = data_dummies['품목대분류코드']
feature = df_M_and_C

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(feature)
feature_scaled = pd.DataFrame(scaler.transform(feature), columns= feature.columns)

In [ ]:
data = pd.concat([target, feature_scaled], axis = 1)
data = pd.melt(data, id_vars= '품목대분류코드', var_name= 'features', value_name= 'value')
fig, ax = plt.subplots(1, 1, figsize = (10, 10))
sns.violinplot(x = 'features', y = 'value', hue = '품목대분류코드', data = data, split = True, inner = 'quart')
plt.xticks(rotation= 90)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred = None, pred_proba = None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    #ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    #ROC-AUC print추가
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1 : {3:.4f}, AUC : {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

y_target = data_dummies['품목대분류코드']
X_features = data_dummies.drop(['품목대분류코드'], axis =1)
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lr_clf = LogisticRegression()


X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size= 0.2, stratify=y_target)

#DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
dt_pred_proba = dt_clf.predict_proba(X_test)[:, 1]
get_clf_eval(y_test, dt_pred, dt_pred_proba)

#RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]
get_clf_eval(y_test, rf_pred, rf_pred_proba)

#LogisticRegression 학습/예측/평가
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
lr_pred_proba = lr_clf.predict_proba(X_test)[:, 1]
get_clf_eval(y_test, lr_pred, lr_pred_proba)

KeyboardInterrupt: 

In [ ]:
def get_model_train_eval(model, ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred, pred_proba)

In [ ]:
from xgboost import XGBClassifier

data = pd.read_csv(r'C:\Users\user\Desktop\경진대회\금융보안원_경진대회_데이터(최종제출).csv')
data = data.drop(['품목대분류명', '품목중분류명'], axis = 1)
data = data.drop(['기준년월', '품목중분류코드', '고객소재지_읍면동'], axis = 1)
data = get_encoding(data, '품목대분류코드', 21)

columns = ['성별', '연령', '가구생애주기', '고객소재지_광역시도', '고객소재지_시군구']

for col in columns:
    data_dummies = get_dummies(col, data)

y_target = data_dummies['품목대분류코드']
X_features = data_dummies.drop(['품목대분류코드'], axis =1)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size= 0.2, stratify=y_target)

xgb_clf = XGBClassifier(n_estimators = 500)
gb_clf = GradientBoostingClassifier()
rf_clf = RandomForestClassifier()

clf_list = [xgb_clf, gb_clf, rf_clf]

for clf in clf_list:
    get_model_train_eval(clf, ftr_train= X_train, ftr_test=X_test, tgt_train=y_train, tgt_test = y_test)


In [ ]:
ftr_importances = pd.Series(dt_clf.feature_importances_, index = X_features.columns)

ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

In [ ]:
#feature importance 추출
print("Feature importance:\{0}".format(np.round(ftr_top20, 3)))

#feature importance를 column 별로 시각화하기
fig, ax = plt.subplots()
sns.barplot(x = ftr_top20, y = ftr_top20.index) 

In [ ]:
def get_outlier_0(df = None, column = None, weight = 1.5, target = None):
    fraud = df[df[target] == 0][column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)

    iqr = quantile_75 - quantile_25
    iqr_weight = iqr*weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight

    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    return outlier_index


def get_outlier_1(df = None, column = None, weight = 1.5, target = None):
    fraud = df[df[target] == 1][column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)

    iqr = quantile_75 - quantile_25
    iqr_weight = iqr*weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight

    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    return outlier_index    

In [ ]:
data = pd.read_csv(r'C:\Users\user\Desktop\경진대회\금융보안원_경진대회_데이터(최종제출).csv')
data = data.drop(['품목대분류명', '품목중분류명'], axis = 1)
data = data.drop(['기준년월', '품목중분류코드', '고객소재지_읍면동'], axis = 1)
data = get_encoding(data, '품목대분류코드', 16)

outlier_index = get_outlier_1(df = data, column='매출금액', weight= 1.5, target = '품목대분류코드')
data.drop(outlier_index, axis = 0, inplace=True)

outlier_index = get_outlier_1(df = data, column='매출건수', weight= 1.5, target = '품목대분류코드')
data.drop(outlier_index, axis = 0, inplace=True)

columns = ['성별', '연령', '가구생애주기', '고객소재지_광역시도', '고객소재지_시군구']

for col in columns:
    data_dummies = get_dummies(col, data)

data_dummies = get_preprocessed_df(data_dummies, '매출금액')
data_dummies = get_preprocessed_df(data_dummies, '매출건수')
print(data_dummies.shape)

In [ ]:
from sklearn.decomposition import PCA


y_target = data_dummies['품목대분류코드']
X_features = data_dummies.drop(['품목대분류코드'], axis =1)

print(y_target.shape)
print(X_features.shape)

n = 8
pca = PCA(n_components= n)
data_pca = pca.fit_transform(X_features)
print(data_pca.shape)

pca_columns = []
for i in range(1, n + 1):
    component = 'pca_component_' + str(i)
    pca_columns.append(component)

df_pca = pd.DataFrame(data_pca, columns= pca_columns)
df_pca['target'] = np.array(y_target)

In [ ]:
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier

X_features = df_pca.drop('target', axis = 1)
y_target = df_pca['target']

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size= 0.2, stratify=y_target)

smote = SMOTE()
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

dt_clf = DecisionTreeClassifier()
lr_clf = LogisticRegression()
lgbm_clf = LGBMClassifier(n_estimators= 1000, num_leaves= 64, n_jobs= -1, boost_from_average = False)

clf_list = [dt_clf, lr_clf, lgbm_clf]

for clf in clf_list:
    get_model_train_eval(clf, ftr_train= X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test = y_test)

In [ ]:
xgb_clf = XGBClassifier(n_estimators = 1000)
xgb_clf.fit(X_train_over, y_train_over, early_stopping_rounds= 200, eval_metric = 'auc', eval_set = [(X_train_over, y_train_over), (X_test, y_test)])

xgb_pred = xgb_clf.predict(X_test)
xgb_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]
get_clf_eval(y_test, xgb_pred, xgb_pred_proba)

In [ ]:
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train_over, y_train_over)
gb_pred = gb_clf.predict(X_test)
gb_pred_proba = gb_clf.predict_proba(X_test)[:, 1]
get_clf_eval(y_test, gb_pred, gb_pred_proba)

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb_clf = XGBClassifier(n_estimators = 100)

params = {
    'max_depth' : [5, 7],
    'min_child_weight' : [1, 3],
    'colsample_bytree' : [0.5, 0.75]
}

grid_cv = GridSearchCV(xgb_clf, param_grid=params, cv = 3, verbose= 1)
grid_cv.fit(X_train_over, y_train_over, early_stopping_rounds = 30, eval_metric = 'auc', eval_set = [(X_train_over, y_train_over), (X_test, y_test)])

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
lgbm_clf = LGBMClassifier(n_estimators= 200)

params = {
    'num_leaves' : [32, 64],
    'max_depth' : [128, 160],
    'min_child_samples' : [60, 100],
    'subsample' : [0.8, 1]
}

gridcv = GridSearchCV(lgbm_clf, param_grid= params, cv = 3)
gridcv.fit(X_train_over, y_train_over, early_stopping_rounds =  30, eval_metric = 'auc', eval_set = [(X_train_over, y_train_over), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)
print('최고 auc: {0:.4f}'.format(roc_auc_score(y_test, gridcv.predict_proba(X_test)[:, 1], average = 'macro')))
print('최고 예측 정확도: {0:.4f}'.format(gridcv.best_score_))